In [1]:
from dotenv import dotenv_values
import openai
from openai.error import RateLimitError
import pandas as pd
import polars as pl
from pprint import pprint
import time
import os
import sys
import krippendorff
from sklearn.metrics import classification_report
import plotly.express as px
from tqdm.auto import tqdm


# Mein OpenAI API Token ist in der .env Datei abgespeichert. Mit folgenden Funktionen lade ich den Token, ohne diesen direkt in diesem Notebook hinterlegen zu müssen.
environment_variables = dotenv_values()
openai.api_key = environment_variables["OPENAI_API_KEY"]

## Annotierte Reddit Kommentare

Meine fertig annotierten Reddit Kommentare lade ich mit der Polars Bibliothek aus einer CSV Datei in ein Polars Dataframe. 

In [2]:
annotated_comments_df = pl.read_csv("datasets/annotated_comments_final.csv")

# Die Spalte mit dem Timestamp "created_utc" muss ich einmalig von einem String in einen richtigen Timestamp umwandeln.
annotated_comments_df = annotated_comments_df.with_columns(pl.col("created_utc").str.strptime(dtype=pl.Datetime))

annotated_comments_df

comment_id,subreddit,title,body,created_utc,comment_url,annotation
i64,str,str,str,datetime[μs],str,str
28118968601,"""de""","""Dilemma""","""Das ist ja gen…",2015-11-15 21:21:53,"""https://www.re…","""na"""
28219776121,"""de""","""15 bekannte Ze…","""""Die Flüchtlin…",2016-01-07 14:26:51,"""https://www.re…","""na"""
28215376812,"""de""","""Rechtsextreme …","""wobei man darü…",2016-01-05 12:55:49,"""https://www.re…","""a"""
28215451441,"""de""","""Rechtsextreme …","""Ich streite ni…",2016-01-05 14:03:53,"""https://www.re…","""s"""
28306497360,"""de""","""Länder des Wes…","""Die Situation …",2016-02-18 20:32:35,"""https://www.re…","""s"""
28201508909,"""de""","""Gemüse und Gem…","""Warum müssen e…",2015-12-29 01:39:25,"""https://www.re…","""na"""
28118443956,"""de""","""Langsam sind a…","""Jap. Zumal auf…",2015-11-15 16:13:22,"""https://www.re…","""na"""
27978086489,"""de""","""Eure Prognose …","""[Bosnienkrieg …",2015-09-03 17:01:48,"""https://www.re…","""na"""
28141803287,"""de""","""Dresdnerin hat…","""wo steht eigen…",2015-11-27 18:18:43,"""https://www.re…","""na"""


# Hilfsfunktionen

Für die weitere Annotation der Kommentare mit der OpenAI API habe ich folgende Hilfsfunktionen programmiert, welche anschließend für jeden Prompt in einer großen Funktion ausgeführt werden.

## Dateneinlesen

Um das Einlesen der Annotationsergebnisse zu testen, erstelle ich an dieser Stelle einmalig ein Beispiel-Dataset und schreibe es als Parquet Datei an den angegebnen Pfad. Die folgenden Funktionen kann ich dann mit diesem Beispiel-Dataset testen.

In [3]:
# Pfad an welchem das Dataset gespeichert wird
example_comments_parquet_path = "datasets/example_comments.parquet"

# Beispieldaten
data = [[1001, "Comment 1001"], [1002, "Comment 1002"], [1002, "Comment 1003"]]

# Mit dem Beispieldaten erstelle ich ein Polars Dataframe
example_comments_df = pl.DataFrame(schema=["comment_id", "comment_text"], data=data)

# und speiche das Dataframe an den Pfad als Parquet Datei ab.
example_comments_df.write_parquet(example_comments_parquet_path)

print("Beispiel-Kommentardatenset: ")
example_comments_df

Beispiel-Kommentardatenset: 


comment_id,comment_text
i64,str
1001,"""Comment 1001"""
1002,"""Comment 1002"""
1002,"""Comment 1003"""


Wenn ich eine Prompt-Evaluation abbreche, möchte ich bei dem Start des nächsten Durchlauf die Kommentare für welche ich bereits Annotation bekommen habe nicht nochmals annotieren lassen. Deshalb habe ich folgende Funktion programmiert, um die comment_ids für gegebenenfalls bereits vorhandene Annotationsergenisse zu erhalten. 

In [4]:
def get_comment_ids_of_previous_annotations(parquet_path):
    # An dem Pfad parquet_path befinden sich eventuell noch keine Ergenisse. Zum Beispiel beim aller ersten Durchlauf einer
    # Prompt-Evaluation. Deshalb muss ich das Einlesen des Dataset in eine Try Except Block setzen. Befindet sich an dem
    # angegebenen Pfad keine Datei gebe ich deshalb eine leere Liste mit keinen comment_ids zurück.
    try:
        prev_comment_ids = pl.read_parquet(parquet_path).select("comment_id").to_series().to_list()
        return prev_comment_ids
    except FileNotFoundError:
        return []


prev_annotated_comment_ids = get_comment_ids_of_previous_annotations(example_comments_parquet_path)

print("Folge comment_ids wurden an dem angegebenen Pfad gefunden: ", prev_annotated_comment_ids)

Folge comment_ids wurden an dem angegebenen Pfad gefunden:  [1001, 1002, 1002]


Bei jedem Prompt-Evaluation Durchlauf möchte ich die Ergenisse an den angegebenen Pfad als Parquet Datei abspeichern. Bereits vorhandene Ergenisse sollen dabei aber nicht überschrieben werden.

In [5]:
def write_annotations_to_comment_dataset(parquet_path, df):
    # An dem angegebenen Pfad befinden sich eventuel bereits Ergenisse.
    try:
        # Wenn das der Fall ist, lese ich die bereits vorhandenen Ergenisse ein und hänge die neuen Daten an.
        prev_annotated_comments_df = pl.read_parquet(parquet_path)
        updated_annotated_comments_df = prev_annotated_comments_df.vstack(df)
        updated_annotated_comments_df.write_parquet(parquet_path)
    except FileNotFoundError:
        # Wenn noch keine vorherigen Ergebnisse vorhanden sind, schreibe ich einfach direkt die neuen Daten an
        # den angegebenen Pfad.
        df.write_parquet(parquet_path)


# Hier erstelle ich ein Polars Dataframe mit neuen Kommentaren, welche den bereits vorhandenen Kommentaren angehangen werden sollen.
new_comments = [[1004, "Comment 1004"], [1005, "Comment 1005"]]
new_comments_df = pl.DataFrame(schema=["comment_id", "comment_text"], data=new_comments)

write_annotations_to_comment_dataset(example_comments_parquet_path, new_comments_df)

print("Beispiel-Dataset mit neuen Kommentaren: ")
updated_dataset = pl.read_parquet(example_comments_parquet_path)
updated_dataset

Beispiel-Dataset mit neuen Kommentaren: 


comment_id,comment_text
i64,str
1001,"""Comment 1001"""
1002,"""Comment 1002"""
1002,"""Comment 1003"""
1004,"""Comment 1004"""
1005,"""Comment 1005"""


## Prompts

Mit den folgenden Hilfsfunktionen kann ich jede Art von Prompt generieren, zum Beispiel Zero Shot oder Few Shot Prompts. Die OpenAI API benötigt für das Model "gpt-3.5-turbo" eine Liste von Dicts. Diese Python Dicts enhalten die "role" (z.B. "system" für den Systemprompt oder "user" für den Userprompt) und den eigentlichen Text des Prompt im Feld "content" des Dict. 

In [6]:
def zero_shot_prompt_messages(system_prompt, comment_text):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": comment_text},
    ]


system_prompt = "Dies ist der System Prompt."
comment_text = "Beispieltext"
prompt_messages = zero_shot_prompt_messages(system_prompt, comment_text)

print("Beispiel Zero Shot Prompt Messages: ")
print(prompt_messages)

Beispiel Zero Shot Prompt Messages: 
[{'role': 'system', 'content': 'Dies ist der System Prompt.'}, {'role': 'user', 'content': 'Beispieltext'}]


In [7]:
def few_shot_prompt_messages(system_prompt, comment_text, few_shot_examples):
    messages = []
    messages.append({"role": "system", "content": system_prompt})

    for user_prompt, assistent_prompt in few_shot_examples:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": assistent_prompt})

    messages.append({"role": "user", "content": comment_text})

    return messages


few_shot_examples = [
    ("Beispiel 01", "Nicht-Zutreffend"),
    ("Beispiel 02", "Anti-Solidarität"),
    ("Beispiel 03", "Solidarität"),
]

prompt_messages = few_shot_prompt_messages(system_prompt, comment_text, few_shot_examples)

print("Beispiel Few Shot Prompt Messages: ")

# Hier nutze ich "Pretty Print", um die Liste der Dicts übersichtlich darzustellen.
# vgl. https://realpython.com/python-pretty-print/
pprint(prompt_messages, width=140)

Beispiel Few Shot Prompt Messages: 
[{'content': 'Dies ist der System Prompt.', 'role': 'system'},
 {'content': 'Beispiel 01', 'role': 'user'},
 {'content': 'Nicht-Zutreffend', 'role': 'assistant'},
 {'content': 'Beispiel 02', 'role': 'user'},
 {'content': 'Anti-Solidarität', 'role': 'assistant'},
 {'content': 'Beispiel 03', 'role': 'user'},
 {'content': 'Solidarität', 'role': 'assistant'},
 {'content': 'Beispieltext', 'role': 'user'}]


In [8]:
def cot_prompt_messages(system_prompt, comment_text, model_tought, annotation_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": comment_text},
        {"role": "assistant", "content": model_tought},
        {"role": "user", "content": annotation_prompt},
    ]


model_tought = "Ich denke folgendes..."
annotation_prompt = "Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3"
prompt_messages = cot_prompt_messages(system_prompt, comment_text, model_tought, annotation_prompt)

print("Beispiel Chain of Thought Prompt Messages: ")

# Hier nutze ich "Pretty Print", um die Liste der Dicts übersichtlich darzustellen.
# vgl. https://realpython.com/python-pretty-print/
pprint(prompt_messages, width=140)

Beispiel Chain of Thought Prompt Messages: 
[{'content': 'Dies ist der System Prompt.', 'role': 'system'},
 {'content': 'Beispieltext', 'role': 'user'},
 {'content': 'Ich denke folgendes...', 'role': 'assistant'},
 {'content': 'Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3', 'role': 'user'}]


In [9]:
def cot_few_shot_THOUGHT_prompt_messages(system_prompt, cot_few_shot_examples, annotation_prompt, comment_text):
    messages = []
    messages.append({"role": "system", "content": system_prompt})

    for example in cot_few_shot_examples:
        messages.append({"role": "user", "content": example["user"]})
        messages.append({"role": "assistant", "content": example["assistant"]})
        messages.append({"role": "user", "content": annotation_prompt})
        messages.append({"role": "assistant", "content": example["annotation"]})

    messages.append({"role": "user", "content": comment_text})
    return messages


system_prompt = "Dies ist der System Prompt."

cot_few_shot_examples = [
    {"user": "Beispiel Kommentar 01", "assistant": "Gedanke 01", "annotation": "Finale Annotation 01"},
    {"user": "Beispiel Kommentar 02", "assistant": "Gedanke 02", "annotation": "Finale Annotation 02"},
    {"user": "Beispiel Kommentar 03", "assistant": "Gedanke 03", "annotation": "Finale Annotation 03"},
]

annotation_prompt = "Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3"
comment_text = "Beispiel Kommentar 04"

prompt_messages = cot_few_shot_THOUGHT_prompt_messages(system_prompt, cot_few_shot_examples, annotation_prompt, comment_text)

print("Beispiel Chain of Thought Prompt Messages um den Thought zu generieren: ")

# Hier nutze ich "Pretty Print", um die Liste der Dicts übersichtlich darzustellen.
# vgl. https://realpython.com/python-pretty-print/
pprint(prompt_messages, width=140)

Beispiel Chain of Thought Prompt Messages um den Thought zu generieren: 
[{'content': 'Dies ist der System Prompt.', 'role': 'system'},
 {'content': 'Beispiel Kommentar 01', 'role': 'user'},
 {'content': 'Gedanke 01', 'role': 'assistant'},
 {'content': 'Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3', 'role': 'user'},
 {'content': 'Finale Annotation 01', 'role': 'assistant'},
 {'content': 'Beispiel Kommentar 02', 'role': 'user'},
 {'content': 'Gedanke 02', 'role': 'assistant'},
 {'content': 'Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3', 'role': 'user'},
 {'content': 'Finale Annotation 02', 'role': 'assistant'},
 {'content': 'Beispiel Kommentar 03', 'role': 'user'},
 {'content': 'Gedanke 03', 'role': 'assistant'},
 {'content': 'Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3', 'role': 'user'},
 {'content': 'Finale Annotation 03', 'role': 'assistant'},
 {'content': 'Beispiel Kommentar 04', 'role': 'user'}]


In [10]:
def cot_few_shot_FINAL_ANNOTATION_prompt_messages(prev_cot_messages, model_thought, annotation_prompt):
    prev_cot_messages.append({"role": "assistant", "content": model_thought})
    prev_cot_messages.append({"role": "user", "content": annotation_prompt})

    return prev_cot_messages


prev_cot_messages = cot_few_shot_THOUGHT_prompt_messages(system_prompt, cot_few_shot_examples, annotation_prompt, comment_text)
model_thought = "Ich denke..."

prompt_messages = cot_few_shot_FINAL_ANNOTATION_prompt_messages(prev_cot_messages, model_thought, annotation_prompt)

print("Beispiel Chain of Thought Prompt Messages um die finale Annotation zu generieren: ")

# Hier nutze ich "Pretty Print", um die Liste der Dicts übersichtlich darzustellen.
# vgl. https://realpython.com/python-pretty-print/
pprint(prompt_messages, width=140)

Beispiel Chain of Thought Prompt Messages um die finale Annotation zu generieren: 
[{'content': 'Dies ist der System Prompt.', 'role': 'system'},
 {'content': 'Beispiel Kommentar 01', 'role': 'user'},
 {'content': 'Gedanke 01', 'role': 'assistant'},
 {'content': 'Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3', 'role': 'user'},
 {'content': 'Finale Annotation 01', 'role': 'assistant'},
 {'content': 'Beispiel Kommentar 02', 'role': 'user'},
 {'content': 'Gedanke 02', 'role': 'assistant'},
 {'content': 'Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3', 'role': 'user'},
 {'content': 'Finale Annotation 02', 'role': 'assistant'},
 {'content': 'Beispiel Kommentar 03', 'role': 'user'},
 {'content': 'Gedanke 03', 'role': 'assistant'},
 {'content': 'Wähle jetzt eine der folgenden Optionen: Option 1, Option 2, Option 3', 'role': 'user'},
 {'content': 'Finale Annotation 03', 'role': 'assistant'},
 {'content': 'Beispiel Kommentar 04', 'role': 'user'},
 

## OpenAI API

Um Anfragen an die OpenAI API zu senden, nutze ich die offizielle OpenAI Python Bibliothek. Ich habe mir folgende Hilfsfunktion programmiert, weil die OpenAI API leider öfter Fehler ausgibt. Bei einem RateLimit Fehler warte ich deshalb für 60 Sekunden und machen danach noch einen Versuch. Bei allen anderen Fehlern warte ich nur 1 Sekunde vor dem nächsten Versuch. Dies mache ich für maximal 20 Versuche pro Anfrage.

vgl. https://callmefred.com/how-to-fix-openai-error-ratelimiterror-the-server-had-an-error/

In [11]:
# "request_timeout" => Braucht die OpenAI API länger als 5 Sekunden, breche ich die Anfrage selber ab, weil
# nach 5 Sekunden die Chance hoch ist, dass die Anfrage sowieso nicht beantwortet wird.
def request_chat_completion(prompt_messages, max_token=30, temperature=0.2, request_timeout=5):
    # Die maximale Anzahl an Versuchen, um eine Antwort von den OpenAI API zu bekommen.
    max_retries = 20

    # Die bisher durchgeführten Versuche.
    num_retries = 0

    # Dies ist eine Endlosschleife, welche solange Anfragen an die OpenAI API sendet, bis ich eine Antwort
    # bekomme, oder die maximale Anzahl an Versuchen (20) überschritten wurde.
    while True:
        # Hier sende ich eine Anfrage an die OpenAI API mit der offiziellen OpenAI Python Bibliothek.
        try:
            return openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0613",
                messages=prompt_messages,
                max_tokens=max_token,
                temperature=temperature,
                request_timeout=request_timeout,
            )

        # Sollte ich das RateLimit überschritten haben, warte ich für 1 Minute (60 Sekunden), damit das RateLimit
        # zurück gesetzt wird.
        except RateLimitError as rate_limit_error:
            print(f"[OPENAI] RateLimit erreicht. Warte für 60 Sekunden vor dem nächsten Versuch. Bisherige Versuche: {num_retries}")

            num_retries = num_retries + 1

            # Wenn die maximale Anzahl an Versuchen überschritten ist, werfe ich eine Exception um die
            # Prompt-Evaluation-Pipeline abzubrechen.
            if num_retries > max_retries:
                raise Exception(f"[OPENAI] Maximale Anzahl an Versuchen überschritten!")

            time.sleep(60)

        # Bei jeder anderen Art von Fehler warte ich nur eine Sekunde und propiere es dann nochmal.
        except Exception as e:
            print(
                f"[OPENAI] Ein Fehler ist in der OpenAI API eingetreten. Warte für 1 Sekunden vor dem nächsten Versuch. Bisherige Versuche: {num_retries}"
            )

            num_retries = num_retries + 1

            # Wenn die maximale Anzahl an Versuchen überschritten ist, werfe ich eine Exception um die
            # Prompt-Evaluation-Pipeline abzubrechen.
            if num_retries > max_retries:
                raise Exception(f"[OPENAI] Maximum number of retries ({max_retries}) exceeded. Aborting request.")

            time.sleep(1)


system_prompt = "Du bist ein hilfreicher Assisten."
user_prompt = "Was ist 2+2?"
prompt_messages = zero_shot_prompt_messages(system_prompt, user_prompt)

# openai_reponse = request_chat_completion(prompt_messages)

# print("OpenAI API Antwort: ")

# pprint(openai_reponse, width=140)

## Prompt Evaluation

Die Hilfsfunktion "prompt_evaluation_run" vereint die anderen Hilfsfunktionen, um in einem Durchlauf für alle Kommentare anhand eines Prompt Annotationen zu generieren.

In [12]:
def prompt_evaluation_run(
    annotated_comments_df,
    prompt_type,
    eval_results_parquet_path,
    system_prompt,
    max_token=30,
    few_shot_examples=[],
    cot_annotation_prompt="",
):
    results = []
    total_tokens_used = 0

    # Hier hole ich mir die "comment_ids" aller bisher annotierter Kommentare.
    prev_annotated_comment_ids = get_comment_ids_of_previous_annotations(eval_results_parquet_path)

    # Um die Annotationen zu erstellen, brauche ich nur die Spalten "comment_id" und "body" (Kommentartext).
    selected_comments_df = annotated_comments_df.select(["comment_id", "body"])

    # Dann führe ich einen Filter aus, damit nur noch für noch nicht annotierte Kommentare Annotationen erstellt werden.
    selected_comments = selected_comments_df.filter(~pl.col("comment_id").is_in(prev_annotated_comment_ids)).to_dicts()

    # Anzahl der noch nicht annotierten Kommentare.
    num_comments = len(selected_comments)

    if num_comments > 0:
        print(f"[PROMPT EVAL RUN] Noch {num_comments} nicht annotierte Kommentare verbleibend.")
    else:
        print(f"[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.")
        # Wenn bereits für alle Kommentare annotation erstellt wurden, gebe ich einfach die
        # bisheringen Ergebnisse zurück.
        return pl.read_parquet(eval_results_parquet_path)

    try:
        # Hier interiere ich über jeden noch nicht annotierten Kommentar und lasse mir gleichzeit eine "Progress Bar" anzeigen.
        for comment in tqdm(selected_comments, desc="Annotating comments"):
            # Aus dem Kommentar Dict hole ich mir den Kommentartext als String
            comment_text = comment["body"]

            if prompt_type == "ZERO_SHOT":
                messages = zero_shot_prompt_messages(
                    system_prompt=system_prompt,
                    comment_text=comment_text,
                )

            elif prompt_type == "FEW_SHOT":
                messages = few_shot_prompt_messages(
                    system_prompt=system_prompt,
                    comment_text=comment_text,
                    few_shot_examples=few_shot_examples,
                )

            elif prompt_type == "CHAIN_OF_THOUGHT":
                # Für einen Chain of Thought Prompt muss ich im ersten Schritt zuerst einen "Thought" generieren.
                cot_thought_messages = zero_shot_prompt_messages(system_prompt, comment_text)

                # Der "Thought" darf auch länger sein. Bis zu 300 Token.
                model_thought_max_token = 300

                # Hier generiere ich den "Thought".
                thought_response = request_chat_completion(
                    prompt_messages=cot_thought_messages, max_token=model_thought_max_token, request_timeout=45
                )

                # Die für die OpenAI Anfrage benötigten Token zähle ich mit.
                total_tokens_used = total_tokens_used + thought_response["usage"]["total_tokens"]

                # Aus der Antwort hole ich mir den "Thought" as String
                model_thought = thought_response["choices"][0]["message"]["content"]

                # und speichere den "Thought" im aktuellen Kommentar Dict ab.
                comment["openai_thought"] = model_thought

                # Mit dem "Thought" kann ich jetzt die Prompt Messages für die finale Annotation erstellen.
                messages = cot_prompt_messages(
                    system_prompt=system_prompt,
                    comment_text=comment_text,
                    model_tought=model_thought,
                    annotation_prompt=cot_annotation_prompt,
                )

            elif prompt_type == "FEW_SHOT_CHAIN_OF_THOUGHT":
                # Für einen Few Shot Chain of Thought Prompt muss ich im ersten Schritt zuerst einen "Thought" generieren.
                few_shot_cot_thought_messages = cot_few_shot_THOUGHT_prompt_messages(
                    system_prompt=system_prompt,
                    cot_few_shot_examples=few_shot_examples,
                    annotation_prompt=cot_annotation_prompt,
                    comment_text=comment_text,
                )

                # Der "Thought" darf auch länger sein. Bis zu 300 Token.
                model_thought_max_token = 300

                # Hier generiere ich den "Thought".
                thought_response = request_chat_completion(
                    prompt_messages=few_shot_cot_thought_messages, max_token=model_thought_max_token, request_timeout=45
                )

                # Die für die OpenAI Anfrage benötigten Token zähle ich mit.
                total_tokens_used = total_tokens_used + thought_response["usage"]["total_tokens"]

                # Aus der Antwort hole ich mir den "Thought" as String
                model_thought = thought_response["choices"][0]["message"]["content"]

                # und speichere den "Thought" im aktuellen Kommentar Dict ab.
                comment["openai_thought"] = model_thought

                # Mit dem "Thought" kann ich jetzt die Prompt Messages für die finale Annotation erstellen.
                messages = cot_few_shot_FINAL_ANNOTATION_prompt_messages(
                    prev_cot_messages=few_shot_cot_thought_messages,
                    model_thought=model_thought,
                    annotation_prompt=cot_annotation_prompt,
                )

            # Hier stelle ich jetzt eine OpenAI API Anfrage für die finale Annotation des Kommentar.
            response = request_chat_completion(prompt_messages=messages, max_token=max_token)

            # Die für die OpenAI Anfrage benötigten Token zähle ich mit.
            total_tokens_used = total_tokens_used + response["usage"]["total_tokens"]

            # Aus der Antwort hole ich mir die finale Annotation as String
            model_annotation = response["choices"][0]["message"]["content"]

            # und speichere die finale Annotation im aktuellen Kommentar Dict ab.
            comment["openai_annotation"] = model_annotation

            # Den annotierten Kommentar füge ich zur späteren Auswerting der "resuls" Liste an.
            results.append(comment)

    # Sollte es zu irgendeinem Fehler komme, fange ich diesen hier ab und gebe den Fehler aus.
    except Exception as e:
        print("[PROMPT EVAL RUN] Ein Fehler ist während der des Prompt Evaluation Durchlauf aufgetreten: ", e)

    # Auch wenn ein Fehler während des Durchlauf auftritt, sollen die bisherigen Ergenisse unbedingt gespeichert werden!
    finally:
        print(f"[PROMPT EVAL RUN] Für den Durchlauf wurden {total_tokens_used} Token verbraucht. (ca. ${(total_tokens_used/1000)*0.002})")

        # Hier speichere ich die aktullen Ergenisse als Parquet Datei an dem für den Prompt angegebenen Pfad.
        results_df = pl.from_dicts(results)
        write_annotations_to_comment_dataset(eval_results_parquet_path, results_df)

        # Die bisherigen Ergebnisse gebe ich am Ende der Funktion immer zurück.
        return pl.read_parquet(eval_results_parquet_path)

## Evaluationsmetriken

In [13]:
metrics_results = {}

### Classification Report (f1-score)

In [14]:
def calcuate_classification_report(df, annotation_columns):
    reliability_data = df.select(annotation_columns).to_numpy().astype("U").T

    y_true = reliability_data[0]
    y_pred = reliability_data[1]

    classification_report_str = classification_report(y_true, y_pred)
    classification_report_dict = classification_report(y_true, y_pred, output_dict=True)

    return classification_report_str, classification_report_dict


data = [
    ["a", "b", "c", "a", "b", "c"],
    ["a", "b", "a", "a", "a", "c"],
]

annotation_col_names = ["y_true", "y_pred"]
df = pl.DataFrame(schema=annotation_col_names, data=data)

classification_report_str, classification_report_dict = calcuate_classification_report(df, annotation_col_names)


print("Classification Report: ")
print(classification_report_str)

Classification Report: 
              precision    recall  f1-score   support

           a       0.50      1.00      0.67         2
           b       1.00      0.50      0.67         2
           c       1.00      0.50      0.67         2

    accuracy                           0.67         6
   macro avg       0.83      0.67      0.67         6
weighted avg       0.83      0.67      0.67         6



### Krippendorff's Alpha

In [15]:
def calculate_krippendorf(df, annotation_columns):
    reliability_data = df.select(annotation_columns).to_numpy().astype("U").T
    try:
        kripps_alpha = krippendorff.alpha(reliability_data=reliability_data, level_of_measurement="nominal")
    except Exception as e:
        print("[METRICS] Error while calculating Krippendorf's alpha: ", e)
        kripps_alpha = -1

    return round(kripps_alpha, 4)


data = [
    ["a", "b", "c", "a", "b", "c"],
    ["a", "b", "a", "a", "a", "c"],
    # ["a", "a", "a", "a", "a", "a"],
    # ["a", "a", "a", "a", "a", "a"],
]
annotation_col_names = ["y_true", "y_pred"]
df = pl.DataFrame(schema=annotation_col_names, data=data)

print("Krippendorf's alpah: ", calculate_krippendorf(df, annotation_col_names))

Krippendorf's alpah:  0.5111


## Umwandeln der Annotationen

Mit folgender Hilfsfunktion kann ich anhand eines "annotation_mappings" die OpenAI Model Annotationen in mein eignes Annotationsschame umwandeln.

In [16]:
def parse_prompt_eval_results(df, annotation_mappings):
    # Ersetzt die vordefinierten "annotation_options" mit meinen eigenen Annotationen.
    for annotation_mapping in annotation_mappings:
        for annotation_option, annotation in list(annotation_mapping.items()):
            df = df.with_columns(
                pl.when(pl.col("openai_annotation").str.starts_with(annotation_option))
                .then(pl.lit(annotation))
                .otherwise(pl.col("openai_annotation"))
                .alias("openai_annotation")
            )

    # Wenn das OpenAI Model etwas anderes ausgegeben hat, kann es nicht automatisch zu einer Annotation gematchet werden. In diesem
    # Fall setze ich die Annotation automatisch auf "na".
    df = df.with_columns(
        pl.when(pl.col("openai_annotation").is_in(pl.Series(list(annotation_mappings[0].values()))))
        .then(pl.col("openai_annotation"))
        .otherwise(pl.lit("na"))
    )

    return df


annotation_mappings = [
    {
        "Solidarität": "s",
        "Anti-Solidarität": "a",
        "Nicht-Zutreffend": "na",
    },
    {
        "Klassifizierung: Solidarität": "s",
        "Klassifizierung: Anti-Solidarität": "a",
        "Klassifizierung: Nicht-Zutreffend": "na",
    },
]

example_prompt_eval_results_df = pl.DataFrame(
    [
        {"comment_id": 28215451441, "openai_thought": "Gedanke 01", "openai_annotation": "Solidarität"},
        {"comment_id": 39124746764, "openai_thought": "Gedanke 02", "openai_annotation": "Klassifizierung: Anti-Solidarität"},
    ]
)


example_parsed_prompt_eval_results_df = parse_prompt_eval_results(example_prompt_eval_results_df, annotation_mappings=annotation_mappings)

print("Die umgewandelten Annotationen: ")
example_parsed_prompt_eval_results_df

Die umgewandelten Annotationen: 


comment_id,openai_thought,openai_annotation
i64,str,str
28215451441,"""Gedanke 01""","""s"""
39124746764,"""Gedanke 02""","""a"""


# Sonstige Hilfsfunktionen

In [17]:
def calculate_evaluation_metrics(df, annotation_columns=["annotation", "openai_annotation"]):
    classification_report_str, classification_report_dict = calcuate_classification_report(df, annotation_columns=annotation_columns)
    kripps_alpha = calculate_krippendorf(df, annotation_columns=annotation_columns)
    return classification_report_str, classification_report_dict, kripps_alpha

In [18]:
def split_results_df_between_timeframes(df):
    df_2015 = df.clone().filter(pl.col("created_utc") < pl.date(2020, 1, 1))
    df_2022 = df.clone().filter(pl.col("created_utc") > pl.date(2020, 1, 1))
    return df_2015, df_2022

## Prompt Pipeline

In [19]:
def prompt_pipeline(
    prompt_name,
    prompt_type,
    eval_results_parquet_path,
    annotated_comments_df,
    system_prompt,
    cot_annotation_prompt="",
    few_shot_examples=[],
):
    # Hier starte ich eine Prompt-Evaluation Durchlauf
    prompt_eval_results_df = prompt_evaluation_run(
        annotated_comments_df=annotated_comments_df,
        prompt_type=prompt_type,
        eval_results_parquet_path=eval_results_parquet_path,
        system_prompt=system_prompt,
        cot_annotation_prompt=cot_annotation_prompt,
        few_shot_examples=few_shot_examples,
    )

    # Join mit original annotierten Kommentar Dataset
    if prompt_type in ["ZERO_SHOT", "FEW_SHOT"]:
        annotation_column_names = ["openai_annotation"]
    elif prompt_type in ["CHAIN_OF_THOUGHT", "FEW_SHOT_CHAIN_OF_THOUGHT"]:
        annotation_column_names = ["openai_annotation", "openai_thought"]

    prompt_eval_results_df = annotated_comments_df.clone().join(
        prompt_eval_results_df.select(["comment_id"] + annotation_column_names), on="comment_id"
    )

    # Ich "groupe" alle OpenAI Model Annotationen und lasse sie nach absteigender Häufigkeit anzeigen.
    grouped_model_annotations = prompt_eval_results_df.groupby("openai_annotation").count().sort("count", descending=True)
    print("\n[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen: ", grouped_model_annotations)

    # Hier wandel ich die OpenAI Model Annotationen in mein eigenes Annotationschema um.
    parsed_prompt_eval_results_df = parse_prompt_eval_results(df=prompt_eval_results_df, annotation_mappings=annotation_mappings)
    print("\n[PIPELINE] Umgewandelte OpenAI Model Annotationen: ", parsed_prompt_eval_results_df.groupby("openai_annotation").count())

    # Für diese umgwandelten Annotationen kann ich mir jetzt die Evaluationsmetriken erstellen.
    classification_report_str, classification_report_dict, kripps_alpha = calculate_evaluation_metrics(parsed_prompt_eval_results_df)
    print(f"\n[PIPELINE] Krippendorf's Alpha: {round(kripps_alpha, 4)}")
    print("\n[PIPELINE] Classification Report:\n\n", classification_report_str)

    # Die Evaluationsmetriken lasse ich mir auch getrennt für die zwei Zeiträume ausgeben.
    df_2015, df_2022 = split_results_df_between_timeframes(parsed_prompt_eval_results_df)

    classification_report_str_2015, _, kripps_alpha_2015 = calculate_evaluation_metrics(df_2015)
    print(f"\n[PIPELINE] Krippendorf's Alpha 2015: {round(kripps_alpha_2015, 4)}")
    print("\n[PIPELINE] Classification Report 2015:\n\n", classification_report_str_2015)

    classification_report_str_2022, _, kripps_alpha_2022 = calculate_evaluation_metrics(df_2022)
    print(f"\n[PIPELINE] Krippendorf's Alpha 2022: {round(kripps_alpha_2022, 4)}")
    print("\n[PIPELINE] Classification Report 2022:\n\n", classification_report_str_2022)

    # Zum Schluss gebe ich immer ein Dict mit den wichtigsten Evaluationsmetriken zurück.
    return {
        "prompt": prompt_name,
        "accuracy": classification_report_dict.get("accuracy"),
        "macro f1-score": classification_report_dict["macro avg"]["f1-score"],
        "kripps_alpha": kripps_alpha,
    }


# example_system_prompt = "Klassifiziere folgenden Social-Media-Kommentar mit Solidarität, Anti-Solidarität, oder Nicht-Zutreffend."
# prompt_name = "00 EXAMPLE TEST RUN"
# prompt_type = "ZERO_SHOT"
# parquet_path = "datasets/results/00_example_run_annotations.parquet"

# prompt_pipeline(
#     prompt_name=prompt_name,
#     prompt_type=prompt_type,
#     eval_results_parquet_path=parquet_path,
#     annotated_comments_df=annotated_comments_df.limit(100),
#     system_prompt=example_system_prompt,
# )

# 01 First Try

In [20]:
prompt_name = "01 First Try"
prompt_type = "ZERO_SHOT"
parquet_path = "datasets/results/01_first_try_annotations.parquet"

prompt_01_first_try = "Annotiere den gegebenen Social-Media-Kommentar zu dem Thema Flüchtlinge, Geflüchtete oder Asyl. Bestimme dabei, ob der Kommentar Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt, oder ob er nicht zutreffend ist."

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_01_first_try,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (681, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Anti-Solidarität gegenüber Flüch… ┆ 85    │
│ Der Kommentar drückt Anti-Solida… ┆ 82    │
│ Solidarität gegenüber Flüchtling… ┆ 41    │
│ Der Kommentar drückt Solidarität… ┆ 31    │
│ …                                 ┆ …     │
│ Der gegebene Kommentar drückt An… ┆ 1     │
│ Der Kommentar drückt Solidarität… ┆ 1     │
│ Der Kommentar drückt Anti-Solida… ┆ 1     │
│ Der gegebene Kommentar drückt An… ┆ 1     │
└───────────────────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ 

# 02 Zero Shot Prompt

In [21]:
prompt_name = "02 Zero Shot Prompt"
prompt_type = "ZERO_SHOT"
parquet_path = "datasets/results/02_zero_shot_annotations.parquet"

prompt_02_zero_shot = """Annotieren Sie den gegebenen Social-Media-Kommentar zu dem Thema Flüchtlinge, Geflüchtete oder Asyl. Bestimmen Sie dabei, ob der Kommentar Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt, oder ob er nicht zutreffend ist. Verwenden Sie dafür die folgenden Leitlinien.

Leitlinien:

Definition von Solidarität:
Die Bereitschaft, die eigenen Ressourcen mit anderen zu teilen, sei es direkt, indem man Geld oder Zeit zur Unterstützung anderer spendet, oder indirekt, indem man den Staat dabei unterstützt, einen Teil der durch Steuern oder Beiträge eingenommenen staatlichen Mittel auszugeben und umzuverteilen.

Allgemeine Regeln:
1.	Berücksichtigen Sie keine Links (urls) bei der Annotation.
2.	Ironie und Sarkasmus in Kommentaren müssen berücksichtigt werden.

Detaillierte Regeln für die Annotation:
1.	Ein Kommentar wird mit Solidarität annotiert, wenn:
a.	Der Kommentar eindeutig den Willen zur Unterstützung, die Bereitschaft zum Teilen von Ressourcen und/oder zum Leisten von Hilfe zeigt.
b.	Der Kommentar eine positive Einstellung und Dankbarkeit gegenüber denjenigen zeigt, die Ressourcen teilen und/oder helfen.
c.	Der Kommentar die Europäische Union als Solidaritätsunion befürwortet.
d.	Der Kommentar die EU dafür kritisiert, dass sie nicht genug tut, um Ressourcen zu teilen und/oder zu helfen.
e.	Der Kommentar selbst Hilfe leistet, indem er Informationen teilt oder zur Hilfe und Unterstützung aufruft. 

2.	Ein Kommentar wird mit Anti-Solidarität annotiert, wenn:
a.	Der Kommentar eindeutig zeigt, dass kein Wille zur Unterstützung, keine Bereitschaft zum Teilen von Ressourcen und/oder keine Absicht zum Leisten von Hilfe besteht.
b.	Der Kommentar vorschlägt, Flüchtlinge/Geflüchtete von Ressourcen auszuschließen, zu denen sie derzeit Zugang haben.
c.	Der Kommentar Tendenzen zum Nationalismus und zum Schließen von Grenzen zeigt.
d.	Der Kommentar Flüchtlinge/Geflüchtete als schädlich und/oder schlecht für die Gesellschaft betrachtet.

3.	Ein Kommentar wird mit Nicht-Zutreffend annotiert, wenn:
a.	Der Kommentar nicht eindeutig mit Solidarität oder Anti-Solidarität annotiert werden kann, weil zusätzliche Informationen fehlen.
b.	Der Kommentar keinen Hinweis auf Solidarität oder Anti-Solidarität zeigt.
c.	Der Kommentar andere Themen als Solidarität oder Anti-Solidarität betrifft.
d.	Der Kommentar nicht verständlich ist (enthält z.B. nur Links)."""

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_02_zero_shot,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (14, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Nicht-Zutreffend                  ┆ 458   │
│ Anti-Solidarität                  ┆ 442   │
│ Solidarität                       ┆ 89    │
│ Der gegebene Kommentar betrifft … ┆ 1     │
│ …                                 ┆ …     │
│ Nicht-Zutreffend. Der Kommentar … ┆ 1     │
│ Es tut mir leid, aber ich kann d… ┆ 1     │
│ Der Kommentar wird mit Solidarit… ┆ 1     │
│ Vielen Dank für die Klarstellung… ┆ 1     │
└───────────────────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ s

# 03 Zero Shot Prompt Re1

In [22]:
prompt_name = "03 Zero Shot Prompt Re1"
prompt_type = "ZERO_SHOT"
parquet_path = "datasets/results/03_zero_shot_re1_annotations.parquet"

prompt_03_zero_shot_re1 = """Klassifizieren Sie den gegebenen Social-Media-Kommentar zu dem Thema Flüchtlinge, Geflüchtete oder Asyl. Bestimmen Sie dabei, ob der Kommentar Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt, oder ob er nicht zutreffend ist. Verwenden Sie dafür die folgenden Leitlinien.

Leitlinien:

Definition von Solidarität:
Die Bereitschaft, die eigenen Ressourcen mit anderen zu teilen, sei es direkt, indem man Geld oder Zeit zur Unterstützung anderer spendet, oder indirekt, indem man den Staat dabei unterstützt, einen Teil der durch Steuern oder Beiträge eingenommenen staatlichen Mittel auszugeben und umzuverteilen.

Allgemeine Regeln:
1.	Berücksichtigen Sie keine Links (urls) bei der Annotation.
2.	Ironie und Sarkasmus in Kommentaren müssen berücksichtigt werden. 

Detaillierte Regeln für die Annotation:
1.	Ein Kommentar wird mit Solidarität annotiert, wenn:
a.	Der Kommentar eindeutig den Willen zur Unterstützung, die Bereitschaft zum Teilen von Ressourcen und/oder zum Leisten von Hilfe zeigt.
b.	Der Kommentar eine positive Einstellung und Dankbarkeit gegenüber denjenigen zeigt, die Ressourcen teilen und/oder helfen.
c.	Der Kommentar die Europäische Union als Solidaritätsunion befürwortet.
d.	Der Kommentar die EU dafür kritisiert, dass sie nicht genug tut, um Ressourcen zu teilen und/oder zu helfen.
e.	Der Kommentar selbst Hilfe leistet, indem er Informationen teilt oder zur Hilfe und Unterstützung aufruft. 

2.	Ein Kommentar wird mit Anti-Solidarität annotiert, wenn:
a.	Der Kommentar eindeutig zeigt, dass kein Wille zur Unterstützung, keine Bereitschaft zum Teilen von Ressourcen und/oder keine Absicht zum Leisten von Hilfe besteht.
b.	Der Kommentar vorschlägt, Flüchtlinge/Geflüchtete von Ressourcen auszuschließen, zu denen sie derzeit Zugang haben.
c.	Der Kommentar Tendenzen zum Nationalismus und zum Schließen von Grenzen zeigt.
d.	Der Kommentar Flüchtlinge/Geflüchtete als schädlich und/oder schlecht für die Gesellschaft betrachtet.

3.	Ein Kommentar wird mit Nicht-Zutreffend annotiert, wenn:
a.	Der Kommentar nicht eindeutig mit Solidarität oder Anti-Solidarität annotiert werden kann, weil zusätzliche Informationen fehlen.
b.	Der Kommentar keinen Hinweis auf Solidarität oder Anti-Solidarität zeigt.
c.	Der Kommentar andere Themen als Solidarität oder Anti-Solidarität betrifft.
d.	Der Kommentar nicht verständlich ist (enthält z.B. nur Links)."""

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_03_zero_shot_re1,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (75, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Nicht-Zutreffend                  ┆ 470   │
│ Anti-Solidarität                  ┆ 323   │
│ Solidarität                       ┆ 130   │
│ Nicht zutreffend                  ┆ 6     │
│ …                                 ┆ …     │
│ Der gegebene Kommentar drückt An… ┆ 1     │
│ Der Kommentar drückt Solidarität… ┆ 1     │
│ Der gegebene Kommentar drückt An… ┆ 1     │
│ Der Kommentar drückt Anti-Solida… ┆ 1     │
└───────────────────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ s

# 04 Zero Shot Prompt Re2

In [23]:
prompt_name = "04 Zero Shot Prompt Re2"
prompt_type = "ZERO_SHOT"
parquet_path = "datasets/results/04_zero_shot_re2_annotations.parquet"

prompt_04_zero_shot_re2 = """Klassifizieren Sie den gegebenen Social-Media-Kommentar zu dem Thema Flüchtlinge, Geflüchtete oder Asyl. Bestimmen Sie dabei, ob der Kommentar Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt, oder ob er nicht zutreffend ist. Verwenden Sie dafür die folgenden Leitlinien.
Leitlinien:
Solidarität ist die Bereitschaft, die eigenen Ressourcen mit anderen zu teilen, sei es direkt, indem man Geld oder Zeit zur Unterstützung anderer spendet, oder indirekt, indem man den Staat dabei unterstützt, einen Teil der durch Steuern oder Beiträge eingenommenen staatlichen Mittel auszugeben und umzuverteilen.
Es gibt drei allgemeine Regeln für die Klassifizierung. Erstens dürfen beim Klassifizieren keine Links (URLs) berücksichtigt werden. Zweitens müssen Ironie und Sarkasmus beim Klassifizieren berücksichtigt werden. Drittens klassifizieren Sie nur mit den Wörtern Solidarität, Anti-Solidarität oder Nicht-Zutreffend. Schreiben Sie nichts anderes.
Für das Klassifizieren in die Klassen Solidarität, Anti-Solidarität und Nicht-Zutreffend gibt es detaillierte Regeln.
Ein Kommentar wird mit Solidarität klassifiziert, wenn der Kommentar eindeutig den Willen zur Unterstützung, die Bereitschaft zum Teilen von Ressourcen und/oder zum Leisten von Hilfe zeigt, der Kommentar eine positive Einstellung und Dankbarkeit gegenüber denjenigen zeigt, die Ressourcen teilen und/oder helfen, der Kommentar die Europäische Union als Solidaritätsunion befürwortet, der Kommentar die EU dafür kritisiert, dass sie nicht genug tut, um Ressourcen zu teilen und/oder zu helfen oder der Kommentar selbst Hilfe leistet, indem er Informationen teilt oder zur Hilfe und Unterstützung aufruft.
Ein Kommentar wird mit Anti-Solidarität klassifiziert, wenn der Kommentar eindeutig zeigt, dass kein Wille zur Unterstützung, keine Bereitschaft zum Teilen von Ressourcen und/oder keine Absicht zum Leisten von Hilfe besteht, der Kommentar vorschlägt, Flüchtlinge/Geflüchtete von Ressourcen auszuschließen, zu denen sie derzeit Zugang haben, der Kommentar Tendenzen zum Nationalismus und zum Schließen von Grenzen zeigt oder der Kommentar Flüchtlinge/Geflüchtete als schädlich und/oder schlecht für die Gesellschaft betrachtet.
Ein Kommentar wird mit Nicht-Zutreffend klassifiziert, wenn der Kommentar nicht eindeutig mit Solidarität oder Anti-Solidarität annotiert werden kann, weil zusätzliche Informationen fehlen, der Kommentar keinen Hinweis auf Solidarität oder Anti-Solidarität zeigt, der Kommentar andere Themen als Solidarität oder Anti-Solidarität betrifft oder der Kommentar nicht verständlich ist (enthält z.B. nur Links)."""

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_04_zero_shot_re2,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (4, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Nicht-Zutreffend                  ┆ 468   │
│ Anti-Solidarität                  ┆ 443   │
│ Solidarität                       ┆ 88    │
│ Entschuldigung, aber ich kann Ih… ┆ 1     │
└───────────────────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ a                 ┆ 443   │
│ na                ┆ 469   │
│ s                 ┆ 88    │
└───────────────────┴───────┘

[PIPELINE] Krippendorf's Alpha: 0.0813

[PIPELINE] Cl

# 05 Zero Shot Persona

In [24]:
prompt_name = "05 Zero Shot Persona"
prompt_type = "ZERO_SHOT"
parquet_path = "datasets/results/05_zero_shot_persona_annotations.parquet"

prompt_05_zero_shot_persona = """Du bist ein wissenschaftlicher Mitarbeiter in einem Forschungsprojekt und hast die Aufgabe, Social-Media-Kommentare zu klassifizieren. Das Forschungsprojekt untersucht die Solidarität gegenüber Flüchtlingen und Geflüchteten in Deutschland. Klassifiziere den gegebenen Social-Media Kommentar zu dem Thema Flüchtlinge, Geflüchtete oder Asyl. Bestimme dabei, ob der Kommentar Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt, oder ob er nicht-zutreffend ist. Verwende dafür die folgenden Leitlinien.

Leitlinien:
Definition von Solidarität:
Die Bereitschaft, die eigenen Ressourcen mit anderen zu teilen, sei es direkt, indem man Geld oder Zeit zur Unterstützung anderer spendet, oder indirekt, indem man den Staat dabei unterstützt, einen Teil der durch Steuern oder Beiträge eingenommenen staatlichen Mittel auszugeben und umzuverteilen.

Allgemeine Regeln:
1.	Berücksichtige keine Links (URLs) bei der Klassifizierung.
2.	Ironie und Sarkasmus in Kommentaren müssen berücksichtigt werden.

Detaillierte Regeln für die Klassifizierung:
1.	Ein Kommentar wird mit Solidarität klassifiziert, wenn:
a.	Der Kommentar eindeutig den Willen zur Unterstützung, die Bereitschaft zum Teilen von Ressourcen und/oder zum Leisten von Hilfe zeigt.
b.	Der Kommentar eine positive Einstellung und Dankbarkeit gegenüber denjenigen zeigt, die Ressourcen teilen und/oder helfen.
c.	Der Kommentar die Europäische Union als Solidaritätsunion befürwortet.
d.	Der Kommentar die EU dafür kritisiert, dass sie nicht genug tut, um Ressourcen zu teilen und/oder zu helfen.
e.	Der Kommentar selbst Hilfe leistet, indem er Informationen teilt oder zur Hilfe und Unterstützung aufruft. 

2.	Ein Kommentar wird mit Anti-Solidarität klassifiziert, wenn:
a.	Der Kommentar eindeutig zeigt, dass kein Wille zur Unterstützung, keine Bereitschaft zum Teilen von Ressourcen und/oder keine Absicht zum Leisten von Hilfe besteht.
b.	Der Kommentar vorschlägt, Flüchtlinge/Geflüchtete von Ressourcen auszuschließen, zu denen sie derzeit Zugang haben.
c.	Der Kommentar Tendenzen zum Nationalismus und zum Schließen von Grenzen zeigt.
d.	Der Kommentar Flüchtlinge/Geflüchtete als schädlich und/oder schlecht für die Gesellschaft betrachtet.

3.	Ein Kommentar wird mit Nicht-Zutreffend klassifiziert, wenn:
a.	Der Kommentar nicht eindeutig mit Solidarität oder Anti-Solidarität annotiert werden kann, weil zusätzliche Informationen fehlen.
b.	Der Kommentar keinen Hinweis auf Solidarität oder Anti-Solidarität zeigt.
c.	Der Kommentar andere Themen als Solidarität oder Anti-Solidarität betrifft.
d.	Der Kommentar nicht verständlich ist (enthält z.B. nur Links)."""

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_05_zero_shot_persona,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (268, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Nicht-Zutreffend                  ┆ 270   │
│ Klassifizierung: Anti-Solidaritä… ┆ 201   │
│ Klassifizierung: Nicht-Zutreffen… ┆ 180   │
│ Solidarität                       ┆ 57    │
│ …                                 ┆ …     │
│ Klassifizierung: Solidarität      ┆ 1     │
│                                   ┆       │
│ Be…                               ┆       │
│ Klassifizierung: Anti-Solidaritä… ┆ 1     │
│ Klassifizierung: Solidarität      ┆ 1     │
│                                   ┆       │
│ Be…                               ┆       │
│ Klassifizierung: Anti-Solidaritä… ┆ 1     │
└────────────────────

# 06 Zero Shot Persona Re1

In [25]:
prompt_name = "06 Zero Shot Persona Re1"
prompt_type = "ZERO_SHOT"
parquet_path = "datasets/results/06_zero_shot_persona_re1_annotations.parquet"

prompt_06_zero_shot_persona_re1 = """Sie sind ein wissenschaftlicher Mitarbeiter in einem Forschungsprojekt und haben die Aufgabe, Social-Media-Kommentare zu klassifizieren. Das Forschungsprojekt untersucht die Solidarität gegenüber Flüchtlingen und Geflüchteten in Deutschland. Klassifizieren Sie den gegebenen Social-Media Kommentar zu dem Thema Flüchtlinge, Geflüchtete oder Asyl. Bestimmen Sie dabei, ob der Kommentar Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt, oder ob er nicht-zutreffend ist. Verwenden Sie dafür die folgenden Leitlinien.

Leitlinien:
Definition von Solidarität:
Die Bereitschaft, die eigenen Ressourcen mit anderen zu teilen, sei es direkt, indem man Geld oder Zeit zur Unterstützung anderer spendet, oder indirekt, indem man den Staat dabei unterstützt, einen Teil der durch Steuern oder Beiträge eingenommenen staatlichen Mittel auszugeben und umzuverteilen.

Allgemeine Regeln:
1.	Berücksichtigen Sie keine Links (URLs) bei der Klassifizierung.
2.	Ironie und Sarkasmus in Kommentaren müssen berücksichtigt werden. 

Detaillierte Regeln für die Klassifizierung:
1.	Ein Kommentar wird mit Solidarität klassifiziert, wenn:
a.	Der Kommentar eindeutig den Willen zur Unterstützung, die Bereitschaft zum Teilen von Ressourcen und/oder zum Leisten von Hilfe zeigt.
b.	Der Kommentar eine positive Einstellung und Dankbarkeit gegenüber denjenigen zeigt, die Ressourcen teilen und/oder helfen.
c.	Der Kommentar die Europäische Union als Solidaritätsunion befürwortet.
d.	Der Kommentar die EU dafür kritisiert, dass sie nicht genug tut, um Ressourcen zu teilen und/oder zu helfen.
e.	Der Kommentar selbst Hilfe leistet, indem er Informationen teilt oder zur Hilfe und Unterstützung aufruft. 

2.	Ein Kommentar wird mit Anti-Solidarität klassifiziert, wenn:
a.	Der Kommentar eindeutig zeigt, dass kein Wille zur Unterstützung, keine Bereitschaft zum Teilen von Ressourcen und/oder keine Absicht zum Leisten von Hilfe besteht.
b.	Der Kommentar vorschlägt, Flüchtlinge/Geflüchtete von Ressourcen auszuschließen, zu denen sie derzeit Zugang haben.
c.	Der Kommentar Tendenzen zum Nationalismus und zum Schließen von Grenzen zeigt.
d.	Der Kommentar Flüchtlinge/Geflüchtete als schädlich und/oder schlecht für die Gesellschaft betrachtet.

3.	Ein Kommentar wird mit Nicht-Zutreffend klassifiziert, wenn:
a.	Der Kommentar nicht eindeutig mit Solidarität oder Anti-Solidarität annotiert werden kann, weil zusätzliche Informationen fehlen.
b.	Der Kommentar keinen Hinweis auf Solidarität oder Anti-Solidarität zeigt.
c.	Der Kommentar andere Themen als Solidarität oder Anti-Solidarität betrifft.
d.	Der Kommentar nicht verständlich ist (enthält z.B. nur Links)."""

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_06_zero_shot_persona_re1,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (328, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Nicht-Zutreffend                  ┆ 229   │
│ Klassifizierung: Nicht-Zutreffen… ┆ 229   │
│ Klassifizierung: Anti-Solidaritä… ┆ 187   │
│ Solidarität                       ┆ 30    │
│ …                                 ┆ …     │
│ Klassifizierung: Solidarität      ┆ 1     │
│                                   ┆       │
│ Be…                               ┆       │
│ Klassifizierung: Nicht-Zutreffen… ┆ 1     │
│ Klassifizierung: Solidarität      ┆ 1     │
│                                   ┆       │
│ Be…                               ┆       │
│ Klassifizierung: Anti-Solidaritä… ┆ 1     │
└────────────────────

# 07 One Shot

In [26]:
prompt_name = "07 One Shot"
prompt_type = "FEW_SHOT"
parquet_path = "datasets/results/07_one_shot_annotations.parquet"

few_shot_examples = [
    (
        "Die meisten Flüchtlinge kommen aber mit der GFK und nicht mit Artikel 16 GG, der ist hier schlicht nicht relevant, sondern hauptsächlich die GFK, die Grundrechtecharta, die UN-Charta und die europäische Menschenrechtskonvention.",
        "Nicht-Zutreffend",
    )
]

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_06_zero_shot_persona_re1,
    few_shot_examples=few_shot_examples,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (5, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Anti-Solidarität                  ┆ 461   │
│ Nicht-Zutreffend                  ┆ 283   │
│ Solidarität                       ┆ 254   │
│ 1. Nicht-Zutreffend               ┆ 1     │
│ 2. Nicht-Zut…                     ┆       │
│ Es tut mir leid, aber ich habe k… ┆ 1     │
└───────────────────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ s                 ┆ 254   │
│ na                ┆ 285   │
│ a                 ┆ 

# 08 Few Shot Demo 3

In [27]:
prompt_name = "08 Few Shot Demo 3"
prompt_type = "FEW_SHOT"
parquet_path = "datasets/results/08_few_shot_demo3_annotations.parquet"

few_shot_examples = [
    (
        "Die meisten Flüchtlinge kommen aber mit der GFK und nicht mit Artikel 16 GG, der ist hier schlicht nicht relevant, sondern hauptsächlich die GFK, die Grundrechtecharta, die UN-Charta und die europäische Menschenrechtskonvention.",
        "Nicht-Zutreffend",
    ),
    (
        "erzähl das Australien, Dänemark, Skandinavien ... 🤦 wenn Deutschland wirklich will kann es. für abgelehnte nur Sachleistungen und Bett wäre ein erster Schritt. die Asylbewerber zahlen würden sich schlagartig halbieren!.",
        "Anti-Solidarität",
    ),
    (
        "Ich hab meinem Rathaus geschrieben, dass ich ein Zimmer für einen oder zwei Flüchtlinge bereitstellen könnte für umsonst. Ka, ob es da richtig ist, aber die werden das wohl weiterleiten.",
        "Solidarität",
    ),
]

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_06_zero_shot_persona_re1,
    few_shot_examples=few_shot_examples,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (9, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Nicht-Zutreffend                  ┆ 489   │
│ Anti-Solidarität                  ┆ 320   │
│ Solidarität                       ┆ 185   │
│ Entschuldigung für das Missverst… ┆ 1     │
│ Nicht-Zutreffend (enthält nur ei… ┆ 1     │
│ Ironie/Sarkasmus - Nicht-Zutreff… ┆ 1     │
│ Nicht-Zutreffend (aufgrund des H… ┆ 1     │
│ Ironie/Sarkasmus                  ┆ 1     │
│ Ironie/Sarkasmus (Nicht-Zutreffe… ┆ 1     │
└───────────────────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ st

# 09 Few Shot Demo 10

In [28]:
prompt_name = "09 Few Shot Demo 10"
prompt_type = "FEW_SHOT"
parquet_path = "datasets/results/09_few_shot_demo10_annotations.parquet"

few_shot_examples = [
    (
        "Naja. Klappt in Australien oder Israel ja auch einigermaßen? Und ich denke die Investitionen um Europa abzuschotten dürften geringer sein als das was die Flüchtlinge an Kosten nach sich ziehen,",
        "Anti-Solidarität",
    ),
    (
        "Was soll ich als Privatperson denn tun? Ich kann keine Boote ins Mittelmeer schicken, um die Flüchtlinge zu retten. Das können nur die Politiker und NGOs. Ich empfinde es aber trotzdem als wichtig, dass die Bevölkerung ein Zeichen setzt, dass Flüchtlinge willkommen sind. Auch wenn das Video die Welt nicht ändert, ist es doch besser, als bei den ganzen Hetztaten stumm zu bleiben.",
        "Solidarität",
    ),
    (
        "Die meisten Flüchtlinge kommen aber mit der GFK und nicht mit Artikel 16 GG, der ist hier schlicht nicht relevant, sondern hauptsächlich die GFK, die Grundrechtecharta, die UN-Charta und die europäische Menschenrechtskonvention.",
        "Nicht-Zutreffend",
    ),
    ("Weiß jemand, wie der Anteil von Flüchtlingen ohne Ausweisdokumente vor 2015 war?", "Nicht-Zutreffend"),
    (
        '"In sozialen Netzwerken hatten zuletzt Äußerungen die Runde gemacht, dass es sich bei den Tätern um Flüchtlinge gehandelt haben soll. Dies sei definitiv nicht der Fall, so der Beamte gegenüber EXPRESS."',
        "Nicht-Zutreffend",
    ),
    (
        "erzähl das Australien, Dänemark, Skandinavien ... 🤦 wenn Deutschland wirklich will kann es. für abgelehnte nur Sachleistungen und Bett wäre ein erster Schritt. die Asylbewerber zahlen würden sich schlagartig halbieren!.",
        "Anti-Solidarität",
    ),
    (
        "Ich hab meinem Rathaus geschrieben, dass ich ein Zimmer für einen oder zwei Flüchtlinge bereitstellen könnte für umsonst. Ka, ob es da richtig ist, aber die werden das wohl weiterleiten.",
        "Solidarität",
    ),
    (
        "Also Venezuela ist richtig arm dran. Meine Frau kommt da her und ich war 3 Wochen über die Feiertage dort. Das kriegt man hier nur nocht richtig mit. Weitaus die meisten Flüchtlinge in der Welt…",
        "Nicht-Zutreffend",
    ),
    (
        "Flüchtlinge aus Ostdeutschland, Preußen und wo se nicht alle herkamen waren schon nicht gern gesehen. Glaube den Deutschen ist egal woher, Hauptsache nicht bei uns",
        "Nicht-Zutreffend",
    ),
    ("Er hat  Fake Masken an Flüchtlinge gespendet er sollte lieber  in den Knast als hier verteidigt zu werden", "Nicht-Zutreffend"),
]

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_06_zero_shot_persona_re1,
    few_shot_examples=few_shot_examples,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (7, 2)
┌───────────────────────────────────┬───────┐
│ openai_annotation                 ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ u32   │
╞═══════════════════════════════════╪═══════╡
│ Nicht-Zutreffend                  ┆ 400   │
│ Anti-Solidarität                  ┆ 359   │
│ Solidarität                       ┆ 236   │
│ Ironie erkannt. Nicht-Zutreffend… ┆ 2     │
│ Bitte beachten Sie, dass ich kei… ┆ 1     │
│ Ironie/Sarkasmus                  ┆ 1     │
│ Entschuldigung, aber ich kann Ih… ┆ 1     │
└───────────────────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ na                ┆ 405   │
│ s   

# 10 CoT Zero Shot

In [29]:
prompt_name = "10 CoT Zero Shot"
prompt_type = "CHAIN_OF_THOUGHT"
parquet_path = "datasets/results/10_cot_zero_shot_annotations.parquet"

prompt_10_cot_zero_shot = """Sie sind ein wissenschaftlicher Mitarbeiter in einem Forschungsprojekt und haben die Aufgabe, Social-Media-Kommentare zu klassifizieren. Das Forschungsprojekt untersucht die Solidarität gegenüber Flüchtlingen und Geflüchteten in Deutschland. Klassifizieren Sie den gegebenen Social-Media Kommentar zu dem Thema Flüchtlinge, Geflüchtete oder Asyl. Bestimmen Sie dabei, ob der Kommentar Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt, oder ob er nicht-zutreffend ist und erklären Sie ihre Entscheidung in zwei bis drei Sätzen. Verwenden Sie dafür die folgenden Leitlinien.

Leitlinien:
Definition von Solidarität:
Die Bereitschaft, die eigenen Ressourcen mit anderen zu teilen, sei es direkt, indem man Geld oder Zeit zur Unterstützung anderer spendet, oder indirekt, indem man den Staat dabei unterstützt, einen Teil der durch Steuern oder Beiträge eingenommenen staatlichen Mittel auszugeben und umzuverteilen.

Allgemeine Regeln:
1.	Berücksichtigen Sie keine Links (URLs) bei der Klassifizierung.
2.	Ironie und Sarkasmus in Kommentaren müssen berücksichtigt werden. 

Detaillierte Regeln für die Klassifizierung:
1.	Ein Kommentar wird mit Solidarität klassifiziert, wenn:
a.	Der Kommentar eindeutig den Willen zur Unterstützung, die Bereitschaft zum Teilen von Ressourcen und/oder zum Leisten von Hilfe zeigt.
b.	Der Kommentar eine positive Einstellung und Dankbarkeit gegenüber denjenigen zeigt, die Ressourcen teilen und/oder helfen.
c.	Der Kommentar die Europäische Union als Solidaritätsunion befürwortet.
d.	Der Kommentar die EU dafür kritisiert, dass sie nicht genug tut, um Ressourcen zu teilen und/oder zu helfen.
e.	Der Kommentar selbst Hilfe leistet, indem er Informationen teilt oder zur Hilfe und Unterstützung aufruft. 

2.	Ein Kommentar wird mit Anti-Solidarität klassifiziert, wenn:
a.	Der Kommentar eindeutig zeigt, dass kein Wille zur Unterstützung, keine Bereitschaft zum Teilen von Ressourcen und/oder keine Absicht zum Leisten von Hilfe besteht.
b.	Der Kommentar vorschlägt, Flüchtlinge/Geflüchtete von Ressourcen auszuschließen, zu denen sie derzeit Zugang haben.
c.	Der Kommentar Tendenzen zum Nationalismus und zum Schließen von Grenzen zeigt.
d.	Der Kommentar Flüchtlinge/Geflüchtete als schädlich und/oder schlecht für die Gesellschaft betrachtet.

3.	Ein Kommentar wird mit Nicht-Zutreffend klassifiziert, wenn:
a.	Der Kommentar nicht eindeutig mit Solidarität oder Anti-Solidarität annotiert werden kann, weil zusätzliche Informationen fehlen.
b.	Der Kommentar keinen Hinweis auf Solidarität oder Anti-Solidarität zeigt.
c.	Der Kommentar andere Themen als Solidarität oder Anti-Solidarität betrifft.
d.	Der Kommentar nicht verständlich ist (enthält z.B. nur Links)."""

prompt_10_cot_zero_shot_final_annotation = (
    "Was ist die finale Annotation? Antworte dieses mal nur mit Solidarität, Anit-Solidarität oder Nicht-Zutreffend."
)

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_10_cot_zero_shot,
    cot_annotation_prompt=prompt_10_cot_zero_shot_final_annotation,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (5, 2)
┌─────────────────────┬───────┐
│ openai_annotation   ┆ count │
│ ---                 ┆ ---   │
│ str                 ┆ u32   │
╞═════════════════════╪═══════╡
│ Anti-Solidarität    ┆ 350   │
│ Nicht-Zutreffend.   ┆ 313   │
│ Nicht-Zutreffend    ┆ 231   │
│ Solidarität         ┆ 105   │
│ 1. Nicht-Zutreffend ┆ 1     │
│ 2. Nicht-Zut…       ┆       │
└─────────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ na                ┆ 545   │
│ a                 ┆ 350   │
│ s                 ┆ 105   │
└───────────────────┴───────┘

[PIPELINE] Krippendorf's Alpha: 0.1547

[PIPELINE] Classification Report:

               precision    recall  f1-score   support

# 11 CoT Few Shot

In [30]:
prompt_name = "11 CoT Few Shot"
prompt_type = "FEW_SHOT_CHAIN_OF_THOUGHT"
parquet_path = "datasets/results/11_cot_few_shot_annotations.parquet"

cot_few_shot_examples = [
    {
        "user": "Naja. Klappt in Australien oder Israel ja auch einigermaßen? Und ich denke die Investitionen um Europa abzuschotten dürften geringer sein als das was die Flüchtlinge an Kosten nach sich ziehen,",
        "assistant": "Der Kommentar schlägt vor, die Grenzen der EU zu schließen und keine Flüchtlinge mehr in Deutschland aufzunehmen, um dadurch Kosten zu sparen. Damit zeigt der Kommentar den Willen zum Schließen von Grenzen und dass keine Bereitschaft zum Teilen von Ressourcen oder Leisten von Hilfe besteht. Er ist mit Anti-Solidarität zu annotieren.",
        "annotation": "Anti-Solidarität",
    },
    {
        "user": "Was soll ich als Privatperson denn tun? Ich kann keine Boote ins Mittelmeer schicken, um die Flüchtlinge zu retten. Das können nur die Politiker und NGOs. Ich empfinde es aber trotzdem als wichtig, dass die Bevölkerung ein Zeichen setzt, dass Flüchtlinge willkommen sind. Auch wenn das Video die Welt nicht ändert, ist es doch besser, als bei den ganzen Hetztaten stumm zu bleiben.",
        "assistant": "Der Kommentar zeigt deutlich, dass der Autor des Kommentars Flüchtlinge in Deutschland willkommen heißen möchte. Dadurch drückt der Kommentar indirekt den Willen zum Teilen von Ressourcen und eindeutig die Bereitschaft zum Leisten von Hilfe aus. Deswegen ist er mit Solidarität zu annotieren.",
        "annotation": "Solidarität",
    },
    {
        "user": "Die meisten Flüchtlinge kommen aber mit der GFK und nicht mit Artikel 16 GG, der ist hier schlicht nicht relevant, sondern hauptsächlich die GFK, die Grundrechtecharta, die UN-Charta und die europäische Menschenrechtskonvention.",
        "assistant": "Der Kommentar stellt eine Sachlage in Bezug auf Flüchtlinge dar, positioniert sich allerdings nicht dazu. Deswegen zeigt der Kommentar weder Solidarität noch Anti-Solidarität und ist nicht-zutreffend.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "Weiß jemand, wie der Anteil von Flüchtlingen ohne Ausweisdokumente vor 2015 war?",
        "assistant": "Der Kommentar stellt eine sachliche Frage in Bezug auf Flüchtlinge. Es gibt weder Hinweise auf Solidarität noch Anti-Solidarität, deswegen ist der Kommentar nicht zutreffend.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": '"In sozialen Netzwerken hatten zuletzt Äußerungen die Runde gemacht, dass es sich bei den Tätern um Flüchtlinge gehandelt haben soll. Dies sei definitiv nicht der Fall, so der Beamte gegenüber EXPRESS."',
        "assistant": "Der Kommentar ist in Anführungsstrichen geschrieben, was darauf hinweist, dass er jemanden oder etwas anderes zitiert und keine eigene Meinung ausdrückt. Zusätzlich stellt der Kommentar lediglich eine Sachlage dar, ohne sich dazu zu positionieren. Deswegen ist der Kommentar nicht-zutreffend.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "erzähl das Australien, Dänemark, Skandinavien ... 🤦 wenn Deutschland wirklich will kann es. für abgelehnte nur Sachleistungen und Bett wäre ein erster Schritt. die Asylbewerber zahlen würden sich schlagartig halbieren!.",
        "assistant": "Der Kommentar schlägt vor, Asylbewerber, welche auch geflüchtete Menschen sind, von Ressourcen auszuschließen, welche ihnen aktuell noch zur Verfügung stehen, um dafür zu sorgen, dass weniger Flüchtlinge nach Deutschland kommen. Damit zeigt der Kommentar eindeutig, dass keine Bereitschaft zum Teilen von Ressourcen oder Leisten von Hilfe besteht und mit Anti-Solidarität zu annotieren ist.",
        "annotation": "Anti-Solidarität",
    },
    {
        "user": "Ich hab meinem Rathaus geschrieben, dass ich ein Zimmer für einen oder zwei Flüchtlinge bereitstellen könnte für umsonst. Ka, ob es da richtig ist, aber die werden das wohl weiterleiten.",
        "assistant": "Der Kommentar zeigt, dass der Autor des Kommentars bereit ist, Flüchtlinge bei sich aufzunehmen und ihnen damit zu helfen. Damit drückt er eindeutig den Willen zum Leisten von Hilfe aus und ist mit Solidarität zu annotieren.",
        "annotation": "Solidarität",
    },
    {
        "user": "Also Venezuela ist richtig arm dran. Meine Frau kommt da her und ich war 3 Wochen über die Feiertage dort. Das kriegt man hier nur nocht richtig mit. Weitaus die meisten Flüchtlinge in der Welt…",
        "assistant": "Der Kommentar ist nicht eindeutig zu verstehen und es kann weder Solidarität noch Anti-Solidarität festgestellt werden. Deswegen ist der Kommentar nicht-zutreffend.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "Flüchtlinge aus Ostdeutschland, Preußen und wo se nicht alle herkamen waren schon nicht gern gesehen. Glaube den Deutschen ist egal woher, Hauptsache nicht bei uns",
        "assistant": "Der Kommentar stellt eine Vermutung darüber an, wie Deutsche im Allgemeinen zu Flüchtlingen stehen, positioniert sich selbst allerdings nicht dazu. Deswegen zeigt der Kommentar weder Solidarität noch Anti-Solidarität und ist nicht-zutreffend.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "Er hat  Fake Masken an Flüchtlinge gespendet er sollte lieber  in den Knast als hier verteidigt zu werden",
        "assistant": "Der Kommentar äußert sich zu jemandem, der ohne weiteren Kontext nicht näher bestimmt werden kann. Er zeigt weder Anzeichen für Solidarität noch Anti-Solidarität und ist deswegen nicht-zutreffend.",
        "annotation": "Nicht-Zutreffend",
    },
]

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_10_cot_zero_shot,
    cot_annotation_prompt=prompt_10_cot_zero_shot_final_annotation,
    few_shot_examples=cot_few_shot_examples,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (4, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ Nicht-Zutreffend  ┆ 574   │
│ Solidarität       ┆ 216   │
│ Anti-Solidarität  ┆ 209   │
│ Ambivalent        ┆ 1     │
└───────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ a                 ┆ 209   │
│ na                ┆ 575   │
│ s                 ┆ 216   │
└───────────────────┴───────┘

[PIPELINE] Krippendorf's Alpha: 0.2356

[PIPELINE] Classification Report:

               precision    recall  f1-score   support

           a       0.22      0.69      0.34        68
          na       0.93     


[PIPELINE] Krippendorf's Alpha 2022: 0.1198

[PIPELINE] Classification Report 2022:

               precision    recall  f1-score   support

           a       0.03      0.40      0.05         5
          na       0.95      0.64      0.77       449
           s       0.28      0.72      0.41        46

    accuracy                           0.65       500
   macro avg       0.42      0.59      0.41       500
weighted avg       0.88      0.65      0.73       500



# 12 CoT Few Shot Model

In [31]:
prompt_name = "12 CoT Few Shot Model"
prompt_type = "FEW_SHOT_CHAIN_OF_THOUGHT"
parquet_path = "datasets/results/12_cot_few_shot_model_annotations.parquet"

cot_few_shot_examples = [
    {
        "user": "Naja. Klappt in Australien oder Israel ja auch einigermaßen? Und ich denke die Investitionen um Europa abzuschotten dürften geringer sein als das was die Flüchtlinge an Kosten nach sich ziehen,",
        "assistant": "Der Kommentar drückt Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten aus, da er vorschlägt, sie von Ressourcen auszuschließen und die Kosten, die mit ihnen verbunden sind, als negativ darstellt.",
        "annotation": "Anti-Solidarität",
    },
    {
        "user": "Was soll ich als Privatperson denn tun? Ich kann keine Boote ins Mittelmeer schicken, um die Flüchtlinge zu retten. Das können nur die Politiker und NGOs. Ich empfinde es aber trotzdem als wichtig, dass die Bevölkerung ein Zeichen setzt, dass Flüchtlinge willkommen sind. Auch wenn das Video die Welt nicht ändert, ist es doch besser, als bei den ganzen Hetztaten stumm zu bleiben.",
        "assistant": "Der Kommentar drückt Solidarität gegenüber Flüchtlingen und Geflüchteten aus, da er den Willen zur Unterstützung und die Bereitschaft zum Zeigen von Solidarität betont, auch wenn die Person selbst keine direkten Maßnahmen ergreifen kann.",
        "annotation": "Solidarität",
    },
    {
        "user": "Die meisten Flüchtlinge kommen aber mit der GFK und nicht mit Artikel 16 GG, der ist hier schlicht nicht relevant, sondern hauptsächlich die GFK, die Grundrechtecharta, die UN-Charta und die europäische Menschenrechtskonvention.",
        "assistant": "Nicht zutreffend. Der Kommentar bezieht sich nicht auf Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten, sondern auf die rechtlichen Grundlagen, die für sie gelten.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "Weiß jemand, wie der Anteil von Flüchtlingen ohne Ausweisdokumente vor 2015 war?",
        "assistant": "Nicht-Zutreffend. Der Kommentar stellt eine Frage zur Statistik über Flüchtlinge ohne Ausweisdokumente vor 2015 und enthält keine Hinweise auf Solidarität oder Anti-Solidarität.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": '"In sozialen Netzwerken hatten zuletzt Äußerungen die Runde gemacht, dass es sich bei den Tätern um Flüchtlinge gehandelt haben soll. Dies sei definitiv nicht der Fall, so der Beamte gegenüber EXPRESS."',
        "assistant": "Der gegebene Kommentar kann nicht eindeutig mit Solidarität oder Anti-Solidarität annotiert werden, da er keine klare Position zu Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten ausdrückt.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "erzähl das Australien, Dänemark, Skandinavien ... 🤦 wenn Deutschland wirklich will kann es. für abgelehnte nur Sachleistungen und Bett wäre ein erster Schritt. die Asylbewerber zahlen würden sich schlagartig halbieren!.",
        "assistant": "Der Kommentar drückt Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten aus, da er vorschlägt, sie von Ressourcen auszuschließen und sie als finanzielle Belastung für die Gesellschaft betrachtet.",
        "annotation": "Anti-Solidarität",
    },
    {
        "user": "Ich hab meinem Rathaus geschrieben, dass ich ein Zimmer für einen oder zwei Flüchtlinge bereitstellen könnte für umsonst. Ka, ob es da richtig ist, aber die werden das wohl weiterleiten.",
        "assistant": "Der Kommentar drückt Solidarität gegenüber Flüchtlingen aus, da der Autor bereit ist, seine Ressourcen zu teilen und Hilfe anzubieten.",
        "annotation": "Solidarität",
    },
    {
        "user": "Also Venezuela ist richtig arm dran. Meine Frau kommt da her und ich war 3 Wochen über die Feiertage dort. Das kriegt man hier nur nocht richtig mit. Weitaus die meisten Flüchtlinge in der Welt…",
        "assistant": "Nicht-Zutreffend. Der Kommentar bezieht sich auf die Situation in Venezuela und erwähnt Flüchtlinge, gibt jedoch keine klare Aussage zur Solidarität oder Anti-Solidarität gegenüber ihnen.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "Flüchtlinge aus Ostdeutschland, Preußen und wo se nicht alle herkamen waren schon nicht gern gesehen. Glaube den Deutschen ist egal woher, Hauptsache nicht bei uns",
        "assistant": "Anti-Solidarität: Der Kommentar zeigt eine ablehnende Haltung gegenüber Flüchtlingen und Geflüchteten und drückt aus, dass es den Deutschen egal ist, woher sie kommen, solange sie nicht in Deutschland sind.",
        "annotation": "Nicht-Zutreffend",
    },
    {
        "user": "Er hat  Fake Masken an Flüchtlinge gespendet er sollte lieber  in den Knast als hier verteidigt zu werden",
        "assistant": "Nicht zutreffend. Der Kommentar äußert sich kritisch über jemanden, der gefälschte Masken an Flüchtlinge gespendet hat, und fordert Konsequenzen. Es wird jedoch keine klare Position zur Solidarität oder Anti-Solidarität gegenüber Flüchtlingen und Geflüchteten eingenommen.",
        "annotation": "Nicht-Zutreffend",
    },
]

metrics_results[prompt_name] = prompt_pipeline(
    prompt_name=prompt_name,
    prompt_type=prompt_type,
    eval_results_parquet_path=parquet_path,
    annotated_comments_df=annotated_comments_df,
    system_prompt=prompt_10_cot_zero_shot,
    cot_annotation_prompt=prompt_10_cot_zero_shot_final_annotation,
    few_shot_examples=cot_few_shot_examples,
)

[PROMPT EVAL RUN] Für alle Kommentare wurden bereits Annotationen erstellt.

[PIPELINE] Nicht umgewandelte OpenAI Model Annotationen:  shape: (4, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ Anti-Solidarität  ┆ 401   │
│ Nicht-Zutreffend  ┆ 399   │
│ Solidarität       ┆ 199   │
│ Ambivalent        ┆ 1     │
└───────────────────┴───────┘

[PIPELINE] Umgewandelte OpenAI Model Annotationen:  shape: (3, 2)
┌───────────────────┬───────┐
│ openai_annotation ┆ count │
│ ---               ┆ ---   │
│ str               ┆ u32   │
╞═══════════════════╪═══════╡
│ na                ┆ 400   │
│ a                 ┆ 401   │
│ s                 ┆ 199   │
└───────────────────┴───────┘

[PIPELINE] Krippendorf's Alpha: 0.0918

[PIPELINE] Classification Report:

               precision    recall  f1-score   support

           a       0.16      0.94      0.27        68
          na       0.95     

# Ergebnisse

In [32]:
metrics_results_pdf = pl.from_dicts(list(metrics_results.values())).to_pandas().set_index("prompt")
metrics_results_pdf

,accuracy,macro f1-score,kripps_alpha
prompt,,,
01 First Try,0.723,0.386563,0.0524
02 Zero Shot Prompt,0.543,0.452561,0.0856
03 Zero Shot Prompt Re1,0.585,0.450146,0.1169
04 Zero Shot Prompt Re2,0.541,0.452150,0.0813
05 Zero Shot Persona,0.605,0.488170,0.1687
06 Zero Shot Persona Re1,0.641,0.503602,0.2021
07 One Shot,0.419,0.389255,0.0065
08 Few Shot Demo 3,0.589,0.484128,0.1762
09 Few Shot Demo 10,0.519,0.442915,0.1112


In [33]:
px.line(metrics_results_pdf, width=800)

# Ergebnisse über 5 Pipeline Runs - 06 Zero Shot Persona Re1

In [34]:
# Für jeden Prompt-Evaluation Durchlauf möchte ich nur die Ergebnisse abspeichern, aber nicht ausgeben. Deshalb
# deaktiviere ich für die Prompt-Evaluation Durchläufe die "print" Funktion.
# vgl. Stackoverflow: https://stackoverflow.com/a/45669280/10050559
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


persona_re1_pipeline_runs = [
    {"name": "06 Zero Shot Persona Re1 v0", "path": "datasets/results/06_zero_shot_persona_re1_annotations.parquet"},
    {"name": "06 Zero Shot Persona Re1 v1", "path": "datasets/results/06_zero_shot_persona_re1_annotations.v01.parquet"},
    {"name": "06 Zero Shot Persona Re1 v2", "path": "datasets/results/06_zero_shot_persona_re1_annotations.v02.parquet"},
    {"name": "06 Zero Shot Persona Re1 v3", "path": "datasets/results/06_zero_shot_persona_re1_annotations.v03.parquet"},
    {"name": "06 Zero Shot Persona Re1 v4", "path": "datasets/results/06_zero_shot_persona_re1_annotations.v04.parquet"},
]

metrics_results_06_zero_shot_persona_re1 = {}

for pipeline_run in persona_re1_pipeline_runs:
    run_name = pipeline_run["name"]
    run_path = pipeline_run["path"]

    with HiddenPrints():
        metrics_results_06_zero_shot_persona_re1[run_name] = prompt_pipeline(
            prompt_name=run_name,
            prompt_type="ZERO_SHOT",
            eval_results_parquet_path=run_path,
            annotated_comments_df=annotated_comments_df,
            system_prompt=prompt_06_zero_shot_persona_re1,
        )

metrics_results_pdf = pl.from_dicts(list(metrics_results_06_zero_shot_persona_re1.values())).to_pandas().set_index("prompt")
metrics_results_pdf

,accuracy,macro f1-score,kripps_alpha
prompt,,,
06 Zero Shot Persona Re1 v0,0.641,0.503602,0.2021
06 Zero Shot Persona Re1 v1,0.644,0.511115,0.2117
06 Zero Shot Persona Re1 v2,0.647,0.505998,0.2066
06 Zero Shot Persona Re1 v3,0.642,0.510884,0.2066
06 Zero Shot Persona Re1 v4,0.638,0.500039,0.1939


In [35]:
px.line(metrics_results_pdf, width=800)